In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

code_dir = os.path.dirname(os.getcwd())
sys.path.append(code_dir)

from glob import glob
import pandas as pd
import numpy as np

import essentia.standard as es

from model.utils.audio_utils import load_audio

import IPython.display as ipd

from matplotlib import pyplot as plt

emb_dir = "../logs/emb/"

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [2]:
def analyze_result(df, idx, hop_size=0.5):

    row = df.iloc[idx]

    seq_len = (row['seq_len']+1) * hop_size

    gt_sequence = load_audio(row["gt_track_path"],
                            seg_start_sec=0.5*row["gt_start_segment"], 
                            seg_length_sec=seq_len,
                            normalize=False)

    pred_sequence = load_audio(row["pred_track_path"],
                            seg_start_sec=0.5*row["pred_start_segment"], 
                            seg_length_sec=seq_len,
                            normalize=False)

    print(f"GT: {os.path.basename(row['gt_track_path'])}")
    print(f"Seg: {row['gt_start_segment']}")
    ipd.display(ipd.Audio(gt_sequence, rate=8000, normalize=False))

    print(f"Pred: {os.path.basename(row['pred_track_path'])}")
    print(f"Seg: {row['pred_start_segment']}")
    ipd.display(ipd.Audio(pred_sequence, rate=8000, normalize=False))

In [3]:
dataset_dir  = "../../datasets/neural-audio-fp-dataset/"
music_dir = os.path.join(dataset_dir, "music")

test_dummy_dir = os.path.join(music_dir, "test-dummy-db-100k-full")
test_dummy_paths = glob(os.path.join(test_dummy_dir, "**/*.wav"), recursive=True)
print(len(test_dummy_paths))

test_query_dir = os.path.join(music_dir, "test-query-db-500-30s")

query_dir = os.path.join(test_query_dir, "query", "snr_00dB_10dB_1s")
query_paths = sorted(glob(os.path.join(query_dir, "**/*.wav"), recursive=True))
print(len(query_paths))

db_dir = os.path.join(test_query_dir, "db")
db_paths = sorted(glob(os.path.join(db_dir, "**/*.wav"), recursive=True))
print(len(db_paths))


93458
500
500


In [29]:
#analysis_path = "/home/oaraz/nextcore/fingerprinting/neural-audio-fp/logs/song_try/fma-random_amp-no_bg_ir_norm-bsz_128/100/float32/analysis.csv"

#model_name = "fma-old_sched-val_shift"
#epoc = 100

#model_name = "640_lamb"
#epoc = 101

model_name = "replication"
epoc = 100

analysis_path = os.path.join("../logs/emb_100k/", model_name, str(epoc), "analysis.csv")

analysis_df = pd.read_csv(analysis_path)
print(analysis_df.shape)
analysis_df.head()

(12000, 7)


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
0,14655,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.716792
1,14655,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.571873
2,14655,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,149,0.460790
3,14655,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.380067
4,14655,11,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.406602


In [30]:
# Correct track
correct_df = analysis_df[analysis_df['gt_track_path'] == analysis_df['pred_track_path']]
print(correct_df.shape[0])
correct_df.head()

8656


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
0,14655,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.716792
1,14655,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.571873
3,14655,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.380067
4,14655,11,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.406602
5,14655,19,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.309247


In [31]:
# False track
false_track_df = analysis_df[analysis_df['gt_track_path'] != analysis_df['pred_track_path']]
print(false_track_df.shape[0])
false_track_df.head()

3344


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
2,14655,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,149,0.460790
6,21662,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,526,0.549976
18,23837,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,369,0.602015
19,23837,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,122,0.232358
20,23837,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,670,0.272074


In [32]:
# Correct track, false segment
c_track_f_segment_df = analysis_df[(analysis_df['gt_track_path'] == analysis_df['pred_track_path']) & ((analysis_df['gt_start_segment'] != analysis_df['pred_start_segment']))]
print(c_track_f_segment_df.shape[0])
c_track_f_segment_df.head()

145


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
103,7189,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,50,/home/oaraz/nextcore/fingerprinting/datasets/n...,49,0.386924
264,25567,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,29,0.870708
265,25567,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,39,0.784958
266,25567,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,39,0.764932
267,25567,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,39,0.643031


In [33]:
c_track_very_f_segment_df = analysis_df[(analysis_df['gt_track_path'] == analysis_df['pred_track_path']) & (np.abs(analysis_df['gt_start_segment'] - analysis_df['pred_start_segment']) > 1)]
print(c_track_very_f_segment_df.shape[0])
c_track_very_f_segment_df.head()

62


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
264,25567,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,29,0.870708
265,25567,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,39,0.784958
266,25567,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,39,0.764932
267,25567,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,39,0.643031
268,25567,11,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,44,0.272574


In [ ]:
analyze_result(false_track_df, 4)

In [34]:
counter = 0
for row in false_track_df.iterrows():
    if os.path.basename(row[1].gt_track_path) == os.path.basename(row[1].pred_track_path):
        counter += 1
print(counter)

2548
